In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint
from scipy.optimize import least_squares, curve_fit

def model_forest_biomass(X, t, *params):
    """Model diferensial untuk dinamika biomassa hutan dengan batasan"""
    Y = np.zeros(7)
    
    # Unpack parameters
    s, alpha, beta1, beta2, rho2, r, v1, v2, theta, \
    lambda_, sigma, phi, phi0, phi1, gamma, pi, \
    phi2, gamma1, rho, rho1, omega, omega1 = params
    
    # Sistem persamaan diferensial
    Y[0] = s*X[0]*(1 - X[0]) - alpha*X[0]*X[1] - beta1*X[0]*X[2] - beta2*X[0]*X[0]*X[4] + rho2*X[0]*X[5]
    Y[1] = r*X[1]*(1 - X[1]/X[0]) - v1*X[1]*X[2] - v2*X[1]*X[4]
    Y[2] = theta*X[2]*(1 - X[2]) + lambda_*X[0]*X[2] - sigma*X[2]*X[1]
    Y[3] = phi*X[2] - phi0*X[3] - phi1*X[3]*X[6]
    Y[4] = gamma*X[2] + pi*X[0]*X[0]*X[4]*(1 - X[4]/5.0) + phi2*X[3] - gamma1*X[4]
    Y[5] = rho*(1 - X[0]) - rho1*X[5] + 0.1*X[6]*X[5]
    Y[6] = omega*X[3] - omega1*X[6]
    
    return Y

def generate_gaussian_params(n_samples=100):
    """Generate synthetic parameter data using Gaussian distribution"""
    np.random.seed(42)
    
    param_means = [
        0.1,  # s_B
        0.2,  # alpha_BW
        0.15, # beta1_BN
        0.1,  # beta2_BkuadratH
        0.15, # rho2_BT
        0.3,  # rBW
        0.1,  # v1_WN
        0.1,  # v2_WH
        0.2,  # thetaN
        0.1,  # lambda_beta1_BN
        0.15, # sigma_NW
        0.2,  # phi_N
        0.1,  # phi0_p
        0.1,  # phi1_EP
        0.15, # gamma_N
        0.1,  # pi_beta2_BkuadratH
        0.1,  # phi2_P
        0.2,  # gamma1_H
        0.3,  # rho
        0.1,  # rho1_T
        0.2,  # omega_P
        0.1   # omega1_E
    ]
    
    param_stds = [mean * 0.2 for mean in param_means]
    
    params_data = np.zeros((n_samples, len(param_means)))
    for i in range(len(param_means)):
        params_data[:, i] = np.maximum(0.01, np.random.normal(
            param_means[i], 
            param_stds[i], 
            n_samples
        ))
    
    param_names = [
        's_B', 'alpha_BW', 'beta1_BN', 'beta2_BkuadratH', 'rho2_BT',
        'rBW', 'v1_WN', 'v2_WH', 'thetaN', 'lambda_beta1_BN',
        'sigma_NW', 'phi_N', 'phi0_p', 'phi1_EP', 'gamma_N',
        'pi_beta2_BkuadratH', 'phi2_P', 'gamma1_H', 'rho',
        'rho1_T', 'omega_P', 'omega1_E'
    ]
    
    return pd.DataFrame(params_data, columns=param_names)

def generate_uniform_params(n_samples=100):
    """Generate synthetic parameter data using Uniform distribution"""
    np.random.seed(42)
    
    # Range parameter berdasarkan batasan fisik dan literatur
    param_ranges = [
        (0.05, 0.15),   # s_B: laju pertumbuhan biomassa
        (0.1, 0.3),     # alpha_BW: laju predasi satwa
        (0.1, 0.2),     # beta1_BN: laju interaksi biomassa-manusia
        (0.05, 0.15),   # beta2_BkuadratH: laju kerusakan oleh aktivitas manusia
        (0.1, 0.2),     # rho2_BT: laju pemulihan oleh teknologi
        (0.2, 0.4),     # rBW: laju pertumbuhan satwa
        (0.05, 0.15),   # v1_WN: laju interaksi satwa-manusia
        (0.05, 0.15),   # v2_WH: laju pengurangan satwa oleh aktivitas
        (0.15, 0.25),   # thetaN: laju pertumbuhan populasi
        (0.05, 0.15),   # lambda_beta1_BN: faktor penguatan interaksi B-N
        (0.1, 0.2),     # sigma_NW: laju interaksi manusia-satwa
        (0.15, 0.25),   # phi_N: laju tekanan populasi
        (0.05, 0.15),   # phi0_p: laju pengurangan tekanan alami
        (0.05, 0.15),   # phi1_EP: laju pengurangan tekanan ekonomi
        (0.1, 0.2),     # gamma_N: laju peningkatan aktivitas oleh populasi
        (0.05, 0.15),   # pi_beta2_BkuadratH: faktor penguatan kerusakan
        (0.05, 0.15),   # phi2_P: laju peningkatan aktivitas oleh tekanan
        (0.15, 0.25),   # gamma1_H: laju pengurangan aktivitas alami
        (0.25, 0.35),   # rho: laju pengembangan teknologi
        (0.05, 0.15),   # rho1_T: laju penurunan efektivitas teknologi
        (0.15, 0.25),   # omega_P: laju respons ekonomi
        (0.05, 0.15)    # omega1_E: laju penurunan efektivitas ekonomi
    ]
    
    params_data = np.zeros((n_samples, len(param_ranges)))
    
    for i in range(len(param_ranges)):
        params_data[:, i] = np.random.uniform(
            param_ranges[i][0],
            param_ranges[i][1],
            n_samples
        )
    
    param_names = [
        's_B', 'alpha_BW', 'beta1_BN', 'beta2_BkuadratH', 'rho2_BT',
        'rBW', 'v1_WN', 'v2_WH', 'thetaN', 'lambda_beta1_BN',
        'sigma_NW', 'phi_N', 'phi0_p', 'phi1_EP', 'gamma_N',
        'pi_beta2_BkuadratH', 'phi2_P', 'gamma1_H', 'rho',
        'rho1_T', 'omega_P', 'omega1_E'
    ]
    
    return pd.DataFrame(params_data, columns=param_names), param_ranges

def objective_function(params, t, observed_data):
    """Fungsi objektif untuk optimasi dengan penanganan error"""
    try:
        X0 = [0.9, 0.3, 0.2, 0.1, 0.05, 0.2, 0.1]
        solution = odeint(model_forest_biomass, X0, t, args=(tuple(params),))
        residuals = solution - observed_data
        residuals[~np.isfinite(residuals)] = 1e6
        return np.ravel(residuals)
    except:
        return np.ones(len(t) * 7) * 1e6

def curve_fit_function(t, *params):
    """Wrapper function for curve_fit"""
    X0 = [0.9, 0.3, 0.2, 0.1, 0.05, 0.2, 0.1]
    solution = odeint(model_forest_biomass, X0, t, args=(params,))
    return np.ravel(solution)

def run_analysis():
    """Fungsi utama untuk menjalankan analisis"""
    t = np.linspace(0, 100, 1000)
    X0 = [0.9, 0.3, 0.2, 0.1, 0.05, 0.2, 0.1]
    
    gaussian_params = generate_gaussian_params(n_samples=100)
    uniform_params, param_ranges = generate_uniform_params(n_samples=100)
    
    results = {}
    for method, params in [("Gaussian", gaussian_params.iloc[0].values),
                           ("Uniform", uniform_params.iloc[0].values)]:
        original_solution = odeint(model_forest_biomass, X0, t, args=(tuple(params),))
        
        np.random.seed(42)
        noise_level = 0.005
        observed_data = np.maximum(0, np.minimum(
            original_solution + noise_level * np.random.randn(*original_solution.shape),
            1.5 * original_solution
        ))
        
        # Fitting using least_squares
        result_ls = least_squares(
            objective_function, 
            params, 
            args=(t, observed_data),
            bounds=([1e-6]*len(params), [10.0]*len(params)),
            method='trf',
            loss='soft_l1',
            ftol=1e-8,
            xtol=1e-8,
            max_nfev=2000
        )
        
        fitted_solution_ls = odeint(model_forest_biomass, X0, t, args=(tuple(result_ls.x),))
        
        # Fitting using curve_fit
        result_cf, _ = curve_fit(
            curve_fit_function, 
            t, 
            np.ravel(observed_data), 
            p0=params, 
            bounds=([1e-6]*len(params), [10.0]*len(params))
        )
        
        fitted_solution_cf = odeint(model_forest_biomass, X0, t, args=(tuple(result_cf),))
        
        # Hitung metrics
        metrics_ls = calculate_fitting_metrics(original_solution, fitted_solution_ls)
        metrics_cf = calculate_fitting_metrics(original_solution, fitted_solution_cf)
        
        # Simpan hasil
        results[method] = {
            'least_squares': {
                'fitted': fitted_solution_ls,
                'metrics': metrics_ls,
                'params': result_ls.x
            },
            'curve_fit': {
                'fitted': fitted_solution_cf,
                'metrics': metrics_cf,
                'params': result_cf
            }
        }
    
    # Print metrics comparison
    print("\nMetrics Comparison:")
    for method in results:
        print(f"\n{method} - Least Squares:")
        for metric, value in results[method]['least_squares']['metrics'].items():
            print(f"{metric}: {value:.6f}")
        
        print(f"\n{method} - Curve Fit:")
        for metric, value in results[method]['curve_fit']['metrics'].items():
            print(f"{metric}: {value:.6f}")

if __name__ == "__main__":
    run_analysis()

ValueError: not enough values to unpack (expected 22, got 1)